In [1]:
from pyiceberg import __version__

__version__


'0.9.1'

In [ ]:
# ## Load NYC Taxi/Limousine Trip Data
# 
# For this notebook, we will use the New York City Taxi and Limousine Commision Trip Record Data that's available on the AWS Open Data Registry. This contains data of trips taken by taxis and for-hire vehicles in New York City. We'll save this into an iceberg table called `taxis`.
# 
# First, load the Parquet file using PyArrow:


In [2]:
import pyarrow.parquet as pq

tbl_taxis = pq.read_table('/home/iceberg/data/yellow_tripdata_2021-04.parquet')
tbl_taxis


FileNotFoundError: /home/iceberg/data/yellow_tripdata_2021-04.parquet

In [ ]:
# ## Creating the table
# 
# Next, create the namespace, and the `taxis` table from the schema that's derived from the Arrow schema:


In [ ]:
from pyiceberg.catalog import load_catalog
from pyiceberg.exceptions import NamespaceAlreadyExistsError

cat = load_catalog('default')

try:
    cat.create_namespace('default')
except NamespaceAlreadyExistsError:
    pass


In [ ]:
from pyiceberg.exceptions import NoSuchTableError

try:
    cat.drop_table('default.taxis')
except NoSuchTableError:
    pass

tbl = cat.create_table(
    'default.taxis',
    schema=tbl_taxis.schema
)

tbl


In [ ]:
# ## Write the actual data into the table
# 
# This will create a new snapshot on the table:


In [ ]:
tbl.overwrite(tbl_taxis)

tbl


In [ ]:
# ## Append more data
# 
# Let's append another month of data to the table:


In [ ]:
tbl.append(pq.read_table('/home/iceberg/data/yellow_tripdata_2021-05.parquet'))
tbl


In [ ]:
# ## Load data into a PyArrow Dataframe
# 
# We'll fetch the table using the REST catalog that comes with the setup.


In [ ]:
tbl = cat.load_table('default.taxis')

sc = tbl.scan(row_filter="tpep_pickup_datetime >= '2021-05-01T00:00:00.000000'")


In [ ]:
df = sc.to_arrow().to_pandas()


In [ ]:
len(df)


In [ ]:
df.info()


In [ ]:
df


In [ ]:
df.hist(column='fare_amount')


In [ ]:
import numpy as np
from scipy import stats

stats.zscore(df['fare_amount'])

# Remove everything larger than 3 stddev
df = df[(np.abs(stats.zscore(df['fare_amount'])) < 3)]
# Remove everything below zero
df = df[df['fare_amount'] > 0]


In [ ]:
df.hist(column='fare_amount')


In [ ]:
# # DuckDB
# 
# Use DuckDB to Query the PyArrow Dataframe directly.


In [2]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


Traceback (most recent call last):
  File "/Users/kratochvmat/proj/my/data_tutorials/.venv/lib/python3.12/site-packages/sql/connection.py", line 45, in __init__
    engine = sqlalchemy.create_engine(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 2, in create_engine
  File "/Users/kratochvmat/proj/my/data_tutorials/.venv/lib/python3.12/site-packages/sqlalchemy/util/deprecations.py", line 281, in warned
    return fn(*args, **kwargs)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^
  File "/Users/kratochvmat/proj/my/data_tutorials/.venv/lib/python3.12/site-packages/sqlalchemy/engine/create.py", line 553, in create_engine
    entrypoint = u._get_entrypoint()
                 ^^^^^^^^^^^^^^^^^^^
  File "/Users/kratochvmat/proj/my/data_tutorials/.venv/lib/python3.12/site-packages/sqlalchemy/engine/url.py", line 772, in _get_entrypoint
    cls = registry.load(name)
          ^^^^^^^^^^^^^^^^^^^
  File "/Users/kratochvmat/proj/my/data_tutorials/.venv/lib/python3.

In [ ]:
%sql SELECT * FROM df LIMIT 20


In [ ]:
%%sql --save tip_amount --no-execute

SELECT tip_amount
FROM df


In [ ]:
%sqlplot histogram --table df --column tip_amount --bins 22 --with tip_amount


In [ ]:
%%sql --save tip_amount_filtered --no-execute

WITH tip_amount_stddev AS (
    SELECT STDDEV_POP(tip_amount) AS tip_amount_stddev
    FROM df
)

SELECT tip_amount
FROM df, tip_amount_stddev
WHERE tip_amount > 0
  AND tip_amount < tip_amount_stddev * 3


In [ ]:
%sqlplot histogram --table tip_amount_filtered --column tip_amount --bins 50 --with tip_amount_filtered



